## Extraction des données par Web scraping

In [ ]:
# Importation de librairies
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import os

In [ ]:
# Fonction pour scraper - depuis le site web TrustPilot - les avis clients d'une entreprise donnée et les sauvegarder dans un fichier CSV
def scrape_save_reviews(url, start_page=1):

    """
    Definition :
    - Scrapes customer reviews across multiple pages from a given company's TrustPilot URL.
    - Saves the scraped reviews into a CSV file.

    Parameters :
    - url (str) : The TrustPilot URL of the company's page to scrape reviews from.
    (e.g. : https://fr.trustpilot.com/review/boursobank.com , https://fr.trustpilot.com/review/fortuneo.fr, https://fr.trustpilot.com/review/hellobank.fr, https://fr.trustpilot.com/review/www.monabanq.com)
    - start_page (int) : The page number to start scraping from. Default is 1.
    
    Returns :
    - A Pandas DataFrame containing the scraped reviews.
    """

    # Récupération de la réponse HTTPS de la page d'une entreprise
    response = requests.get(url)

    if response.status_code == 200:
        
        # Création d'un objet BeautifulSoup à partir du contenu HTML récupéré
        soup = bs(response.text, 'lxml')

        # Récupération du nombre de pages
        total_pages = int(soup.find('a', attrs={'name': 'pagination-button-last'}).text.strip())

        # Création d'un dictionnaire avec différentes variables
        data = {
            'pseudo': [],
            'publication_date': [],
            'rating': [],
            'title': [],
            'review': [],
            'experience_date': [],
            'total_reviews': []
            #'country': []
        }
        
        # Boucle sur les pages
        for page in range(start_page, total_pages + 1):

            # Récupération de la réponse HTTPS de la n-ième page
            response = requests.get(url + '?page=' + str(page))

            if response.status_code == 200:

                # Création d'un objet BeautifulSoup à partir du contenu HTML récupéré
                soup = bs(response.text, 'lxml')

                # Récupération des avis
                reviews = soup.find_all('div', class_='styles_cardWrapper__kOLEb styles_show__qAseP')

                # Boucle sur les avis
                for review in reviews:
        
                    # Pseudo du client
                    pseudo_element = review.find('span', class_='typography_heading-xs__osRhC typography_appearance-default__t8iAq')
                    data['pseudo'].append(pseudo_element.text.strip() if pseudo_element else '')

                    # Date de publication de l'avis du client
                    publication_date_element = review.find('time')['datetime']
                    data['publication_date'].append(publication_date_element.strip() if publication_date_element else '')

                    # Note liée à l'avis du client
                    rating_element = review.find('div', class_='styles_reviewHeader__PuHBd')['data-service-review-rating']
                    data['rating'].append(rating_element.strip() if rating_element else '')
                 
                    # Titre de l'avis du client
                    title_element = review.find('h2', class_='typography_heading-xs__osRhC typography_appearance-default__t8iAq')
                    data['title'].append(title_element.text.strip() if title_element else '')
                
                    # Texte de l'avis du client
                    review_element = review.find('p', class_='typography_body-l__v5JLj typography_appearance-default__t8iAq')
                    data['review'].append(review_element.text.strip() if review_element else '')

                    # Date de l'expérience du client
                    experience_date_element = review.find('p', class_='typography_body-m__k2UI7 typography_appearance-default__t8iAq').find('span')
                    data['experience_date'].append(experience_date_element.text.strip() if experience_date_element else '')
                    
                    # Nombre d'avis du client
                    total_reviews_element = review.find('div', class_='styles_consumerExtraDetails__TylYM')['data-consumer-reviews-count']
                    data['total_reviews'].append(total_reviews_element.strip() if total_reviews_element else '')
                    
                    # Pays du client
                    #country_element = review.find('div', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua')
                    #data['country'].append(country_element.text.strip() if country_element else '')
                    
            else:
                print('Erreur lors de la récupération de la page', page, ': erreur', response.status_code)
                break

        # Création d'un DataFrame à partir du dictionnaire "data"
        df = pd.DataFrame(data)

        # Récupération du nom de la banque dans l'URL et Ajout d'une colonne "bank" dans le DataFrame
        match = re.search(r'/review/(?:www\.)?(.*?)\.', url)
        df['bank'] = match.group(1)

        # Sauvegarde du DataFrame dans un fichier CSV
        df.to_csv('../data/raw/scraped_reviews_' + match.group(1) + '_pages_' + str(start_page) + '_to_' + str(page-1) + '.csv', index=False)

        return df
  
    else:
        print('Erreur lors de la récupération de la 1re page', ': erreur', response.status_code)

In [ ]:
# Scraping des avis clients d'une entreprise donnée et Sauvegarde dans un fichier CSV

# >>> ATTENTION ! <<< des fichiers CSV existent déjà dans "data/raw/"

%%time
#df = scrape_save_reviews('https://fr.trustpilot.com/review/www.monabanq.com', 1)

# Affichage des 5 premières lignes
df.head()

UsageError: Line magic function `%%time` not found.


In [ ]:
# Fonction pour concaténer des fichiers CSV et les sauvegarder dans un fichier CSV unique
def concat_save_csv_files(file_prefix):

    """
    Definition :
    - Concatenates CSV files.
    - Saves the concatenated CSV files into a CSV file.

    Parameters :
    - file_prefix (str) : The prefix of the CSV file.
    (e.g. : scraped_reviews)
    
    Returns :
    - A Pandas DataFrame containing the concatenated CSV files.
    """

    # Récupération des fichiers dont le nom comporte le préfixe et qui se termine par un nombre suivi de l'extension ".csv"
    files = [file for file in os.listdir('../data/raw/') if file.startswith(file_prefix) and re.search(r'\d+\.csv$', file)]

    # Création d'une liste de DataFrames en lisant chaque fichier CSV
    dfs = [pd.read_csv(os.path.join('../data/raw/', file)) for file in files]
    
    # Concaténation des DataFrames à partir de la liste de DataFrames "dfs"
    dfs_concatenated = pd.concat(dfs, ignore_index=True)

    # Sauvegarde du DataFrame dans un fichier CSV
    dfs_concatenated.to_csv('../data/raw/scraped_reviews_final.csv', index=False)
    
    return dfs_concatenated

In [ ]:
# Concaténation des fichiers CSV et Sauvegarde dans un fichier CSV unique

# >>> ATTENTION ! <<< un fichier CSV existe déjà dans "data/raw/"

%%time
#dfs_concatenated = concat_save_csv_files('scraped_reviews')

# Affichage des 5 premières lignes
dfs_concatenated.head()

CPU times: user 733 ms, sys: 77.8 ms, total: 811 ms
Wall time: 817 ms


,pseudo,publication_date,rating,title,review,experience_date,total_reviews,bank
0,GERARD CATALA,2025-03-07T13:48:17.000Z,5,Banque pas chère,Banque pas chère,07 mars 2025,13,boursobank
1,JFD,2025-03-07T13:45:35.000Z,5,Modif plafond retrait,"Gestion aisée de mon compte. Pas de frais, auc...",07 mars 2025,2,boursobank
2,Catherine B,2025-03-07T13:35:24.000Z,5,Facilité,Rajouter un bénéficiaire et tout s'est bien dé...,07 mars 2025,1,boursobank
3,Amir B,2025-03-07T13:20:03.000Z,4,Trop d'étape de sécurité,J'ai mis 4 étoiles parce que je trouve qu'il a...,01 mars 2025,1,boursobank
4,Fontaine Joël,2025-03-07T12:28:32.000Z,4,Je recommande cette carte pour les…,Je recommande cette carte pour les voyageurs e...,06 mars 2025,6,boursobank


In [ ]:
dfs_concatenated.shape

(55198, 8)